### IMPORT LIBRARIES

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import skimage.io
import os 
import tqdm
import glob
import tensorflow 

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn import metrics

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.color import grey2rgb

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.utils import to_categorical
from keras import optimizers

from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K

#import tensorflow_addons as tfa
#from tensorflow.keras.metrics import Metric
#from tensorflow_addons.utils.types import AcceptableDTypes, FloatTensorLike
from typeguard import typechecked
from typing import Optional

In [3]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [14]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   validation_split = 0.2,
                                  
        rotation_range=5,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        #zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2)

test_datagen  = ImageDataGenerator(rescale = 1./255
                                  )

In [15]:
data_dir = '/kaggle/input/segmentationmri/Proposed'
image_paths = []
labels = []

for img_file in os.listdir(data_dir):
    full_path = os.path.join(data_dir, img_file)
    if 'cancer' in img_file.lower():
        labels.append(0)  # Class 0: Cancer
    elif 'notcancer' in img_file.lower():
        labels.append(1)  # Class 1: NotCancer
    else:
        continue
    image_paths.append(full_path)

In [4]:
import os

data_dir = '/kaggle/input/segmentationmri/Proposed'
image_paths = []
labels = []

print("Files in directory:", os.listdir(data_dir))  # See all files

for img_file in os.listdir(data_dir):
    full_path = os.path.join(data_dir, img_file)
    if 'Cancer' in img_file.lower():
        labels.append(0)
        image_paths.append(full_path)
        print(f"Added Cancer: {img_file}")
    elif 'NotCancer' in img_file.lower():
        labels.append(1)
        image_paths.append(full_path)
        print(f"Added NotCancer: {img_file}")
    else:
        print(f"Skipped: {img_file}")

print(f"Total images: {len(image_paths)}")
print(f"Label distribution: {np.bincount(labels) if labels else 'Empty'}")

Files in directory: ['Cancer', 'NotCancer']
Skipped: Cancer
Skipped: NotCancer
Total images: 0
Label distribution: Empty


In [5]:
data_dir = '/kaggle/input/segmentationmri/Proposed'
image_paths = []
labels = []
class_names = sorted(os.listdir(data_dir))

print("Subfolders:", class_names)

if len(class_names) != 2:
    raise ValueError(f"Expected 2 subfolders, found {len(class_names)}: {class_names}")

for class_name in class_names:
    class_path = os.path.join(data_dir, class_name)
    for img_file in os.listdir(class_path):
        full_path = os.path.join(class_path, img_file)
        image_paths.append(full_path)
        labels.append(class_names.index(class_name))

image_paths = np.array(image_paths)
labels = np.array(labels)

train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

# Rest of the code as above

Subfolders: ['Cancer', 'NotCancer']


In [6]:
data_dir = '/kaggle/input/segmentationmri/Proposed'
image_paths = []
labels = []
class_names = sorted(os.listdir(data_dir))

print("Subfolders:", class_names)

if len(class_names) != 2:
    raise ValueError(f"Expected 2 subfolders, found {len(class_names)}: {class_names}")

for class_name in class_names:
    class_path = os.path.join(data_dir, class_name)
    for img_file in os.listdir(class_path):
        full_path = os.path.join(class_path, img_file)
        image_paths.append(full_path)
        labels.append(class_names.index(class_name))

image_paths = np.array(image_paths)
labels = np.array(labels)

train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

# Rest of the code (data generators and model) same as above
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_df = pd.DataFrame({'filename': train_paths, 'class': train_labels.astype(str)})
test_df = pd.DataFrame({'filename': test_paths, 'class': test_labels.astype(str)})

train_dataset = train_datagen.flow_from_dataframe(
    train_df, target_size=(224, 224), color_mode='rgb', class_mode='binary', batch_size=32
)
test_dataset = test_datagen.flow_from_dataframe(
    test_df, target_size=(224, 224), color_mode='rgb', class_mode='binary', batch_size=32
)



Subfolders: ['Cancer', 'NotCancer']
Found 3604 validated image filenames belonging to 2 classes.
Found 901 validated image filenames belonging to 2 classes.


In [7]:
# Import necessary libraries  
import numpy as np  
import pandas as pd  
import os  
from keras.models import Model  
from keras.layers import GlobalAveragePooling2D, Dense  
from keras.preprocessing.image import ImageDataGenerator  
from tensorflow.keras.applications import InceptionResNetV2  # Correct import for InceptionResNetV2  
from sklearn.model_selection import train_test_split  


from keras.applications import InceptionResNetV2  

# Load the model without pre-trained weights  
base_model = InceptionResNetV2(input_shape=(224, 224, 3), weights=None, include_top=False)  

# Load weights that you uploaded  
base_model.load_weights('/kaggle/input/ca/keras/default/1/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5')
x = base_model.output  
x = GlobalAveragePooling2D()(x)  
x = Dense(128, activation='relu')(x)  
output = Dense(1, activation='sigmoid')(x)  
model = Model(inputs=base_model.input, outputs=output)  

# Freeze the base model layers  
for layer in base_model.layers:  
    layer.trainable = False  

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  
#model.summary()

In [8]:
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset)
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.4f}")

Epoch 1/10
113/113 [==============================] - 573s 5s/step - loss: 0.5605 - accuracy: 0.7525 - val_loss: 0.4178 - val_accuracy: 0.8313
Epoch 2/10
113/113 [==============================] - 571s 5s/step - loss: 0.3730 - accuracy: 0.8460 - val_loss: 0.3836 - val_accuracy: 0.8491
Epoch 3/10
113/113 [==============================] - 568s 5s/step - loss: 0.3244 - accuracy: 0.8640 - val_loss: 0.3651 - val_accuracy: 0.8557
Epoch 4/10
113/113 [==============================] - 569s 5s/step - loss: 0.3198 - accuracy: 0.8676 - val_loss: 0.3631 - val_accuracy: 0.8479
Epoch 5/10
113/113 [==============================] - 567s 5s/step - loss: 0.2630 - accuracy: 0.8976 - val_loss: 0.3494 - val_accuracy: 0.8513
Epoch 6/10
113/113 [==============================] - 565s 5s/step - loss: 0.2479 - accuracy: 0.9040 - val_loss: 0.3438 - val_accuracy: 0.8535
Epoch 7/10
113/113 [==============================] - 567s 5s/step - loss: 0.2624 - accuracy: 0.8923 - val_loss: 0.3661 - val_accuracy: 0.8557

In [ ]:
import numpy as np  
import pandas as pd  
from sklearn.metrics import f1_score  
from sklearn.model_selection import train_test_split  
from keras.preprocessing.image import ImageDataGenerator  
from tensorflow.keras.applications import InceptionResNetV2  
from keras.models import Model 
from keras.layers import GlobalAveragePooling2D, Dense  
from keras.callbacks import ModelCheckpoint  
# Save training results to a CSV file  
history_df = pd.DataFrame(history.history)  
history_df.to_csv('training_results.csv', index=True)  



In [ ]:
test_preds_prob = model.predict(test_dataset)  
test_preds = (test_preds_prob > 0.5).astype(int)  

# Calculate and display the F1 score  
f1 = f1_score(test_labels, test_preds)  
print(f"F1 Score: {f1:.4f}")  